In [1]:
# Load libraries
import pandas as pd
import bokeh
import numpy as np
from bokeh.palettes import Viridis, Category10, Inferno, TolRainbow, Spectral
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
output_notebook()
pd.options.mode.chained_assignment = None

Loading BokehJS ...

In [2]:
# read in data
uscities = pd.read_csv("uscities.csv") #Source: https://simplemaps.com/data/us-cities
fortune = pd.read_excel("Fortune 500 Data.xlsx") # Source: https://www.kaggle.com/datasets/rm1000/fortune-500-companies?resource=download

In [3]:
# Standardize Names of Cities in both datasets
uscities['city']=uscities['city'].str.upper()
fortune['city']=fortune['headquarters_city'].str.upper()

In [4]:
# combine cities and states 
uscities['citystate']=uscities['city']+uscities['state_id']
fortune['citystate']=fortune['city']+fortune['headquarters_state']
# add count variable
fortune['count']=1

In [5]:
# only keep the columns needed for maps
uscities = uscities[['citystate', 'lat', 'lng']]

In [6]:
# convert latitude and longitude for Mercator
import math

def latlon_to_mercator(lat, lng):

    r_major = 6378137.0 # Earth's major radius in meters
    x = r_major * math.radians(lng)
    y = r_major * math.log(math.tan(math.pi/4 + math.radians(lat)/2))

    return x, y

def convert_df_to_mercator(df, lat_col='lat', lng_col='lng'):

    df['mercator_x'], df['mercator_y'] = zip(*df.apply(lambda row: latlon_to_mercator(row[lat_col], row[lng_col]), axis=1))
    return df

In [7]:
lat_lng = uscities[['lat', 'lng']]
df = convert_df_to_mercator(lat_lng)

In [8]:
df = df[['mercator_x', 'mercator_y']]
uscities = pd.concat([uscities, df], axis=1)
# merge the geodata back onto the broker dealer data 
fortune_data = fortune.merge(uscities, on='citystate', how='left')

In [9]:
fortune_data

,name,rank,year,industry,sector,headquarters_state,headquarters_city,market_value_mil,revenue_mil,profit_mil,...,female_ceo,newcomer_to_fortune_500,global_500,city,citystate,count,lat,lng,mercator_x,mercator_y
0,Walmart,1,2023,General Merchandisers,Retailing,AR,Bentonville,397475.0,611289.0,11680.0,...,no,no,yes,BENTONVILLE,BENTONVILLEAR,1,36.3547,-94.2305,-1.048969e+07,4.349538e+06
1,Amazon,2,2023,Internet Services and Retailing,Retailing,WA,Seattle,1058440.0,513983.0,2722.0,...,no,no,yes,SEATTLE,SEATTLEWA,1,47.6211,-122.3244,-1.361709e+07,6.044049e+06
2,Exxon Mobil,3,2023,Petroleum Refining,Energy,TX,Irving,446424.0,413680.0,55740.0,...,no,no,yes,IRVING,IRVINGTX,1,32.8583,-96.9702,-1.079467e+07,3.876511e+06
3,Apple,4,2023,"Computers, Office Equipment",Technology,CA,Cupertino,2609039.0,394328.0,99803.0,...,no,no,yes,CUPERTINO,CUPERTINOCA,1,37.3168,-122.0466,-1.358617e+07,4.483357e+06
4,UnitedHealth Group,5,2023,Health Care: Insurance and Managed Care,Health Care,MN,Minnetonka,440854.0,324162.0,20120.0,...,no,no,yes,MINNETONKA,MINNETONKAMN,1,44.9322,-93.4598,-1.040390e+07,5.610854e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13935,USG Corporation,496,1996,"Building Materials, Glass",NaN,IL,NaN,NaN,2444.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
13936,Cotter & Co.,497,1996,Specialty Retailers,NaN,IL,NaN,NaN,2437.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
13937,Ace Hardware Corporation,498,1996,Wholesalers,NaN,IL,NaN,NaN,2436.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
13938,General Instrument Corporation,499,1996,"Electronics, Electrical Equipment",NaN,IL,NaN,NaN,2432.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN


In [10]:
fortune_data['market_value']=fortune_data['market_value_mil']*1000000
fortune_data['revenue']=fortune_data['revenue_mil']*1000000

In [11]:
# Make map using 2023 Fortune 500 Data
fortune_2023 = fortune_data[fortune_data['year']==2023]

In [12]:
fortune_2023['Currency']=fortune_2023['market_value'].map("${:,.0f}".format)
fortune_2023['Log Market Value']=np.log(fortune_2023['market_value'].apply(pd.to_numeric))
palette = TolRainbow[5]

tooltips = [("City", "@city"), ("State", "@headquarters_state"), ("Company", "@name")
            ,("Market Value", "@Currency")]

# Define color mapper
color_mapper = linear_cmap(field_name='Log Market Value', palette=palette, 
                low=fortune_2023['Log Market Value'].min(), high=fortune_2023['Log Market Value'].max())

# Create Map object
m = figure(title='2023 Fortune 500 by Company (Colored by Log Market Value)',
           width=650, height=400, x_range=(fortune_2023['mercator_x'].min()-800000,
            fortune_2023['mercator_x'].max()+800000), 
           y_range=(fortune_2023['mercator_y'].min(), fortune_2023['mercator_y'].max()),
           x_axis_type = 'mercator', y_axis_type = 'mercator', tooltips=tooltips)

# adding tile
m.add_tile("CartoDB Positron", retina=True)
m.scatter(x='mercator_x', y='mercator_y', size=5, source=fortune_2023, color= color_mapper,
         fill_alpha=1)

# define color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                    formatter=NumeralTickFormatter(format='0.0[0000]'),
                    label_standoff = 13, width=8, location=(0,0))

# set color_bar location
m.add_layout(color_bar, 'right')

show(m)

In [13]:
# Look at market value and count by city
Value_by_city = fortune_2023.groupby(['city', 'headquarters_state']).agg({'count':'sum',
        'market_value':'sum' ,'mercator_x':'mean', 'mercator_y':'mean'}).reset_index()

In [14]:
# replace infinity and nan with zero
Value_by_city['market_value'] = Value_by_city['market_value'].replace([-np.Inf, np.Inf, np.nan], 0)

In [15]:
# drop if market value is 0
Value_by_city = Value_by_city[Value_by_city['market_value']!=0]

In [16]:
# plot on a map of United States
Value_by_city['Currency']=Value_by_city['market_value'].map("${:,.0f}".format)
Value_by_city['sizes']=np.log(Value_by_city['count'].apply(pd.to_numeric)+1)*4
Value_by_city['Log Market Value']=np.log(Value_by_city['market_value'].apply(pd.to_numeric))
palette = Viridis[7]

tooltips = [("City", "@city"), ("Count", "@count"), ("Market Value", "@Currency")]

# Define color mapper
color_mapper = linear_cmap(field_name='Log Market Value', palette=palette, 
                low=Value_by_city['Log Market Value'].min(), high=Value_by_city['Log Market Value'].max())

# Create Map object
m = figure(title='2023 Fortune 500 by US City (Sized by Log Count, Colored by Log Market Value)',
           width=650, height=400, x_range=(Value_by_city['mercator_x'].min()-800000,
            Value_by_city['mercator_x'].max()+800000), 
           y_range=(Value_by_city['mercator_y'].min(), Value_by_city['mercator_y'].max()),
           x_axis_type = 'mercator', y_axis_type = 'mercator', tooltips=tooltips)

# adding tile
m.add_tile("CartoDB Positron", retina=True)
m.scatter(x='mercator_x', y='mercator_y', size='sizes', source=Value_by_city, color= color_mapper,
         fill_alpha=.9)

# define color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                    formatter=NumeralTickFormatter(format='0.0[0000]'),
                    label_standoff = 13, width=8, location=(0,0))

# set color_bar location
m.add_layout(color_bar, 'right')

show(m)

In [17]:
# Make a scatterplot of 2023 Data
from bokeh.palettes import Viridis256
from bokeh.models import ColumnDataSource, HoverTool
fortune_2023['Log Revenue']=np.log(fortune_2023['revenue'].apply(pd.to_numeric))
fortune_2023['Currency2']=fortune_2023['revenue'].map("${:,.0f}".format)

# Define tooltip
Tooltip = HoverTool(tooltips=[
    ("Company", "@name"),
    ("Market Value", "@a"), 
    ("Revenue", "@b")
])

Tools = 'pan, box_zoom, reset'
p = figure(title='2023 Fortune 500 Market Value vs Revenue', x_axis_label='Log Market Value',
          y_axis_label = 'Log Revenue', tools=Tools)

x=fortune_2023['Log Market Value']
y=fortune_2023['Log Revenue']
name=fortune_2023['name']
a=fortune_2023['Currency']
b=fortune_2023['Currency2']

source = ColumnDataSource(data=dict(x=x, y=y, name=name, a=a, b=b))

color_mapper = LinearColorMapper(palette='Viridis256', low=min(y), high=max(y))

p.scatter(source=source, size=7, fill_color={'field' : 'y', 'transform' : color_mapper})
p.add_tools(Tooltip)

show(p)

In [18]:
# Look at Top Ten Ranked Fortune 500 in 2023 Over Time
top_ten = fortune_data.head(10)
top_ten = top_ten['name'].unique()
top_ten

array(['Walmart', 'Amazon', 'Exxon Mobil', 'Apple', 'UnitedHealth Group',
       'CVS Health', 'Berkshire Hathaway', 'Alphabet', 'McKesson',
       'Chevron'], dtype=object)

In [19]:
Top_Ten = fortune_data[fortune_data['name'].isin(top_ten)]
Top_Ten

,name,rank,year,industry,sector,headquarters_state,headquarters_city,market_value_mil,revenue_mil,profit_mil,...,global_500,city,citystate,count,lat,lng,mercator_x,mercator_y,market_value,revenue
0,Walmart,1,2023,General Merchandisers,Retailing,AR,Bentonville,397475.0,611289.0,11680.0,...,yes,BENTONVILLE,BENTONVILLEAR,1,36.3547,-94.2305,-1.048969e+07,4.349538e+06,3.974750e+11,6.112890e+11
1,Amazon,2,2023,Internet Services and Retailing,Retailing,WA,Seattle,1058440.0,513983.0,2722.0,...,yes,SEATTLE,SEATTLEWA,1,47.6211,-122.3244,-1.361709e+07,6.044049e+06,1.058440e+12,5.139830e+11
2,Exxon Mobil,3,2023,Petroleum Refining,Energy,TX,Irving,446424.0,413680.0,55740.0,...,yes,IRVING,IRVINGTX,1,32.8583,-96.9702,-1.079467e+07,3.876511e+06,4.464240e+11,4.136800e+11
3,Apple,4,2023,"Computers, Office Equipment",Technology,CA,Cupertino,2609039.0,394328.0,99803.0,...,yes,CUPERTINO,CUPERTINOCA,1,37.3168,-122.0466,-1.358617e+07,4.483357e+06,2.609039e+12,3.943280e+11
4,UnitedHealth Group,5,2023,Health Care: Insurance and Managed Care,Health Care,MN,Minnetonka,440854.0,324162.0,20120.0,...,yes,MINNETONKA,MINNETONKAMN,1,44.9322,-93.4598,-1.040390e+07,5.610854e+06,4.408540e+11,3.241620e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4502,Chevron,3,2014,Petroleum Refining,NaN,CA,NaN,227014.0,220356.0,21423.0,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,2.270140e+11,2.203560e+11
4503,Berkshire Hathaway,4,2014,Insurance: Property and Casualty (Stock),NaN,NE,NaN,308003.0,182150.0,19476.0,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,3.080030e+11,1.821500e+11
4504,Apple,5,2014,"Computers, Office Equipment",NaN,CA,NaN,479069.0,170910.0,37037.0,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,4.790690e+11,1.709100e+11
4513,UnitedHealth Group,14,2014,Health Care: Insurance and Managed Care,NaN,MN,NaN,81103.0,122489.0,5625.0,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,8.110300e+10,1.224890e+11


In [20]:
# Pivot the data
Top_Ten = Top_Ten.pivot(index='year', columns='name', values='market_value')
Top_Ten = Top_Ten.reset_index()
Top_Ten = Top_Ten.replace([-np.Inf, np.Inf, np.nan], 0)
Top_Ten

name,year,Alphabet,Amazon,Apple,Berkshire Hathaway,CVS Health,Chevron,Exxon Mobil,McKesson,UnitedHealth Group,Walmart
0,2014,0.000000e+00,0.000000e+00,4.790690e+11,3.080030e+11,0.000000e+00,2.270140e+11,4.220980e+11,4.063300e+10,8.110300e+10,0.000000e+00
1,2015,3.775420e+11,0.000000e+00,7.247730e+11,3.573440e+11,1.171710e+11,1.973810e+11,3.565490e+11,5.266900e+10,1.128130e+11,2.653440e+11
2,2016,5.251190e+11,0.000000e+00,6.043040e+11,3.502790e+11,1.139470e+11,1.796530e+11,3.471290e+11,3.594500e+10,1.225420e+11,2.153560e+11
3,2017,5.794260e+11,0.000000e+00,7.537180e+11,4.110350e+11,8.131000e+10,2.032630e+11,3.400560e+11,3.143900e+10,1.577930e+11,2.186190e+11
4,2018,7.191240e+11,0.000000e+00,8.513180e+11,4.920080e+11,6.311400e+10,2.178450e+11,3.161570e+11,2.906700e+10,2.070800e+11,2.635630e+11
5,2019,8.168240e+11,0.000000e+00,8.956670e+11,4.938700e+11,6.995200e+10,2.340500e+11,3.421720e+11,2.245500e+10,2.372560e+11,2.798800e+11
6,2020,7.989050e+11,9.706800e+11,1.112641e+12,4.428970e+11,7.737600e+10,1.361760e+11,1.606960e+11,2.184500e+10,2.365550e+11,3.218030e+11
7,2021,1.392562e+12,1.558070e+12,2.050666e+12,5.878230e+11,9.865300e+10,2.018650e+11,2.363550e+11,3.104400e+10,3.517250e+11,3.826430e+11
8,2022,1.842326e+12,1.658807e+12,2.849538e+12,7.795420e+11,1.328390e+11,3.171200e+11,3.496520e+11,4.585800e+10,4.798300e+11,4.097950e+11
9,2023,1.330201e+12,1.058440e+12,2.609039e+12,6.756570e+11,9.542200e+10,3.110930e+11,4.464240e+11,4.875700e+10,4.408540e+11,3.974750e+11


In [21]:
# Create the scatterplot with pivoted data
cols = Spectral[10]
Top_Ten['year'] = Top_Ten['year'].astype(str)
top_ten = ['Walmart', 'Amazon', 'Exxon Mobil', 'Apple', 'UnitedHealth Group',
       'CVS Health', 'Berkshire Hathaway', 'Alphabet', 'McKesson',
       'Chevron']

tooltips = [
    ("Name", "$name"),
    ("Market Value", "@$name{$0.0 a}"),
]
fig = figure(x_range=Top_Ten['year'],
            title="Top Ten Fortune 500 Companies' Market Value Over Time", width = 650, 
            tools=["hover"], tooltips=tooltips, y_range=[0,10000000000000])

fig.vbar_stack(top_ten,
              x='year',
              source=Top_Ten,
              color=cols,
              width=.8, legend_label=top_ten)

fig.yaxis.formatter=NumeralTickFormatter(format="$,0")
fig.outline_line_color='black'
fig.legend.location = "top_left"
fig.legend.glyph_height = 12
fig.legend.glyph_width = 12
show(fig)